In [1]:
#import neccessary libraries
import numpy as np
import evaluation as ev
import csv
import pandas as pd

In [2]:
#set root file name
root_file = 'Gamma-E100-n1000-part5'

In [3]:
#read cluster distribution csv as a dataframe and get noise level that minimizes cluster number
df = pd.read_csv('cluster-noise-distribution_'+root_file+'.csv')
noise = df['noise'][df['n_clusters_mean'].idxmin()]

#set run parameters
n_events = 1
event_n = 0

In [4]:
#modifiedtest_single_photon function (taken from test_stats.py) that returns relevant info for each particle, as
#seen in the plot output
#(x, y, z, t, e, clusterindex, pdgid, E_bound, sum(E_hit)
#this function calls our model and tests it using the root file defined in datasets.py in the single_photon_dataset function
def particle_info(n_events, event_n, noise):
    tbeta = .2
    td = .5
    nmax = n_events
    yielder = ev.TestYielderSinglePhoton()
    yielder.model.signal_threshold = noise
    for i, (event, prediction, clustering) in enumerate(yielder.iter_clustering(tbeta, td, nmax=nmax)):
        if(i==event_n):
            np.set_printoptions(suppress=True) #suppress scientific notation
            _, idx, _ = np.unique(clustering, return_counts=True, return_inverse=True)
            energy_values = np.round(np.bincount(idx, event.energy),3)
            return event.xhit, event.yhit, event.zhit, event.energy, event.time, clustering, event.truth_pdgid, event.truth_e_bound, energy_values, np.unique(clustering)
        else:
            continue

In [5]:
#get 5 vectors for all particles of a single event
p_5vector = particle_info(n_events, event_n, noise)

  0%|                                                     | 0/1 [00:17<?, ?it/s]


In [6]:
#write down the 5vector as a pandas dataframe and save it as a csv file
df_5v = pd.DataFrame({'x': np.round(p_5vector[0].astype(np.float64), 2), 'y': np.round(p_5vector[1].astype(np.float64), 2), 
                      'z': np.round(p_5vector[2].astype(np.float64), 2), 'e': np.round(p_5vector[3].astype(np.float64), 3), 
                      't': p_5vector[4].astype(np.float64), 'clusterindex': p_5vector[5], 'pdgid': p_5vector[6].astype(np.int64),
                     'E_bound':np.round(p_5vector[7].astype(np.float64), 3)})
df_5v[df_5v['x']==66.58]

,x,y,z,e,t,clusterindex,pdgid,E_bound
19217,66.58,25.97,453.3,0.047,-1.0,0.0,0,0.0


In [7]:
#write particle info dataframe to .csv 
df_5v.to_csv('particle-info_'+root_file+'_noise-'+str(noise)+'_eventn-'+str(event_n)+'.csv', index=False)

In [ ]:
np.unique(p_5vector[5])